EDA 참고
- 언어의 대부분이 영어 or 포르투갈


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import cosine_similarity

view_log_train = pd.read_csv('/content/drive/MyDrive/commit/commit_github_esaa/project/24-1/view_log.csv')
article_info = pd.read_csv('/content/drive/MyDrive/commit/commit_github_esaa/project/24-1/article_info.csv')
submission = pd.read_csv('/content/drive/MyDrive/commit/commit_github_esaa/project/24-1/sample_submission.csv')





In [ ]:
!pip install nltk

In [ ]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import string
# NLTK 데이터 다운로드
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
article_info['Language'].unique()

array(['en', 'pt', 'es', 'la', 'ja'], dtype=object)

전처리

In [ ]:
def preprocess_text(text, language):
    # 소문자 변환
    text = text.lower()

    # 구두점 제거
    text = text.translate(str.maketrans('', '', string.punctuation))

    try:
        # 언어에 맞는 스탑워드 설정
        stop_words = set(stopwords.words(language))
    except OSError:
        stop_words = set()

    # 스탑워드 제거
    text = ' '.join([word for word in text.split() if word not in stop_words])

    return text

# 언어 코드 변환 사전
language_map = {
    'en': 'english',
    'pt': 'portuguese',
    'es': 'spanish',
    'la': 'latin',  # nltk에는 latin 스탑워드가 없으므로 기본적으로 스탑워드 제거 없음
    'ja': 'japanese'  # nltk에는 japanese 스탑워드가 없으므로 기본적으로 스탑워드 제거 없음
}

# 기사의 언어에 따라 텍스트 전처리 적용
article_info['processed_content'] = article_info.apply(
    lambda row: preprocess_text(row['Content'], language=language_map.get(row['Language'], 'english')), axis=1)

print(article_info[['Content', 'processed_content']].head())



                                             Content  \
0  I've been using git full time for the past 4 y...   
1  Intel has acquired computer vision and machine...   
2  One of the reasons AngularJS is so great to wo...   
3  Despite recent positive news and a renewed int...   
4  Last year around this time, I wrote that The B...   

                                   processed_content  
0  ive using git full time past 4 years wanted sh...  
1  intel acquired computer vision machine learnin...  
2  one reasons angularjs great work developed aro...  
3  despite recent positive news renewed interest ...  
4  last year around time wrote big reverse web wo...  


센텐스버트 사용 + 메타 정보 같이 인코딩

In [ ]:
# SBERT 모델 로드
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
pred_embeddings = model.encode(train.Language+" "+train.userCountry+" "+train.userRegion+" "+train.Title)

In [ ]:
# 사용자-기사 행렬 생성
user_article_matrix = view_log_train.groupby(['userID', 'articleID']).size().unstack(fill_value=0)

# 사용자 간의 유사성 계산
user_similarity = cosine_similarity(user_article_matrix)

# 추천 점수 계산
user_predicted_scores = user_similarity.dot(user_article_matrix) / np.array([np.abs(user_similarity).sum(axis=1)]).T

# 이미 조회한 기사 포함해서 추천
recommendations = []
for idx, user in enumerate(user_article_matrix.index):
    # 해당 사용자의 추천 점수 (높은 점수부터 정렬)
    sorted_indices = user_predicted_scores[idx].argsort()[::-1]
    top5recommend = [user_article_matrix.columns[i] for i in sorted_indices[:5]]

    for article in top5recommend:
        recommendations.append([user, article])

# sample_submission.csv 형태로 DataFrame 생성
top_recommendations = pd.DataFrame(recommendations, columns=['userID', 'articleID'])

submission['articleID'] = top_recommendations['articleID']

submission.to_csv('baseline_submission.csv', index=False)